### Here in this notebook I will forecast to 2021 using Deep learning RNN and LSTM for time series
### The special case of RNN Time series is RNN takes batches from the time series and make a train at each one!
### for example if we have a series from 0 to 9 : [0,1,2,3,4,5,6,7,8,9]
### frist we will devide it into baches :
![](https://scontent.faly3-1.fna.fbcdn.net/v/t1.0-9/121419725_824808158330800_8470839773498296736_n.jpg?_nc_cat=104&_nc_sid=730e14&_nc_ohc=f7BpQHJqLzUAX-JslIO&_nc_ht=scontent.faly3-1.fna&oh=7dfec2af80009b6043a0d429fa299da2&oe=5FA60AB5)

### Second we take bach 1: [ 0,1,2,3] to predict 4
### and bach 2 we take : [1,2,3,4] to predict 5 ...

### For the forecasting it will be like :
![](https://scontent.faly3-1.fna.fbcdn.net/v/t1.0-9/121238276_824808138330802_5818175079055679020_o.jpg?_nc_cat=106&_nc_sid=730e14&_nc_ohc=vbaF1YMJ0YsAX-qt-ZS&_nc_ht=scontent.faly3-1.fna&oh=8ff20dc08795ce0496593038023356ca&oe=5FA68116)

#### 1- Importing:

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

#### 2- Load data

In [ ]:
df= pd.read_csv('../input/real-manufacturing-and-trade-inventories-2020/INVCMRMT.csv', index_col='DATE', parse_dates=True)
df['INVCMRMT']=df['INVCMRMT'].astype(int)
df.index.freq= 'MS'
df['INVCMRMT'].plot(figsize=(12,6)) 

#### 3- Train/ Test split:

In [ ]:
len(df) #We will grap the last year for forecasting

train= df.iloc[:265]
test = df.iloc[265:]

#### 4- Scaling data :

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler= MinMaxScaler()
scaler.fit(train)
scaled_train= scaler.transform(train)
scaled_test= scaler.transform(test)

#### 5- Create Timesereies Generator

In [ ]:
len(test)

In [ ]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
length= 12 #batch size should be smaller than test size
generator= TimeseriesGenerator(scaled_train,scaled_train,length=length,batch_size=1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM
n_features=1 # 1 variable and 1 col 'INVCMRMT'

model= Sequential()
model.add(LSTM(400,activation='relu', input_shape=(length,n_features))) #input shape of batch size
model.add(Dense(1))
model.compile(optimizer='adam',loss='mse')
model.summary()

#### 6- Create Early stop:

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=2) #patience is number of epochs with no improvment
validation_generator= TimeseriesGenerator(scaled_test,scaled_test,length=length,batch_size=1)
model.fit_generator(generator,epochs=10,validation_data=validation_generator,callbacks=early_stop)

#### 7- Loss Evaluation:

In [ ]:
losses= pd.DataFrame(model.history.history)
losses.plot()

- Grap test

In [ ]:
test_predictions = []

first_eval_batch = scaled_train[-length:]
current_batch = first_eval_batch.reshape((1, length, n_features))

for i in range(len(test)):
    
    # get prediction 1 time stamp ahead ([0] is for grabbing just the number instead of [array])
    current_pred = model.predict(current_batch)[0]
    
    # store prediction
    test_predictions.append(current_pred) 
    
    # update batch to now include prediction and drop first value
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
# IGNORE WARNINGS
true_predictions = scaler.inverse_transform(test_predictions)
test['Predictions'] = true_predictions
test.plot(figsize=(12,8))

- Grap predections

In [ ]:
test_predictions = []

first_eval_batch = scaled_train[-length:]
current_batch = first_eval_batch.reshape((1, length, n_features))

for i in range(len(test)):
    
    # get prediction 1 time stamp ahead ([0] is for grabbing just the number instead of [array])
    current_pred = model.predict(current_batch)[0]
    
    # store prediction
    test_predictions.append(current_pred) 
    
    # update batch to now include prediction and drop first value
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
true_predictions = scaler.inverse_transform(test_predictions)
test['Predictions'] = true_predictions

In [ ]:
test

In [ ]:
test.plot(figsize=(12,8))

#### 8- Forecasting

In [ ]:
full_scaler= MinMaxScaler()
scaled_full_data=full_scaler.fit_transform(df)

In [ ]:
length=12
generator=TimeseriesGenerator(scaled_full_data,scaled_full_data,length=length,batch_size=1)
model= Sequential()
model.add(LSTM(400,activation='relu', input_shape=(length,n_features))) #input shape of batch size
model.add(Dense(1))
model.compile(optimizer='adam',loss='mse')


In [ ]:
model.fit_generator(generator,epochs=6)

In [ ]:
forecast = []
# Replace periods with whatever forecast length you want
periods = 12

first_eval_batch = scaled_full_data[-length:]
current_batch = first_eval_batch.reshape((1, length, n_features))

for i in range(periods):
    
    # get prediction 1 time stamp ahead ([0] is for grabbing just the number instead of [array])
    current_pred = model.predict(current_batch)[0]
    
    # store prediction
    forecast.append(current_pred) 
    
    # update batch to now include prediction and drop first value
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
forecast = scaler.inverse_transform(forecast)

 - Creating new time series for forecast

In [ ]:
df

In [ ]:
forecast_index = pd.date_range(start='2020-08-01',periods=periods,freq='MS')
forecast_df = pd.DataFrame(data=forecast,index=forecast_index,columns=['Forecast'])

In [ ]:
forecast_df

- plot the forecast

In [ ]:
ax = df.plot()
forecast_df.plot(ax=ax)

In [ ]:
ax = df.plot()
forecast_df.plot(ax=ax)
plt.xlim('2018-08-01','2021-08-01')